In [3]:
import requests

In [110]:
class Mouse:
    def __init__(self, token="", url="http://127.0.0.1:8801/api/v1"):
        self.token = token
        self.url = url
        size = 16
        self.maze = [[0 for _ in range(size)] for _ in range(size)]
        self.data: dict

    def forward(self):
        r = requests.post(self.url + "/robot-cells/forward", params={"token": self.token})
        return r

    def left(self):
        r = requests.post(self.url + "/robot-cells/left", params={"token": self.token})
        return r

    def right(self):
        r = requests.post(self.url + "/robot-cells/right", params={"token": self.token})
        return r

    def backward(self):
        r = requests.post(self.url + "/robot-cells/backward", params={"token": self.token})
        return r

    def upd_sensor_data(self):
        r = requests.get(self.url + "/robot-cells/sensor-data", params={"token": self.token})
        self.data = r.json()
        return r

    def restart(self):
        r = requests.post(self.url + "/robot-cells/restart", params={"token": self.token})
        return r

    @staticmethod
    def __dist_to_wall(dist: int):
        return int(round((dist - 50) / 166.67))

    def get_walls(self):
        f = self.__dist_to_wall(self.data["front_distance"])
        r = self.__dist_to_wall(self.data["right_side_distance"])
        l = self.__dist_to_wall(self.data["left_side_distance"])
        b = self.__dist_to_wall(self.data["back_distance"])
        return f, r, l, b

In [111]:
m = Mouse()
print(m.forward().text)
print(m.upd_sensor_data().text)

{"detail":"Not Found"}
{"Feedback":true}
{"front_distance":49.48,"right_side_distance":382.14,"left_side_distance":59.33,"back_distance":49.72,"left_45_distance":69.75,"right_45_distance":70.75,"rotation_pitch":-0.27,"rotation_yaw":0.0,"rotation_roll":-0.13,"down_x_offset":-1245.87,"down_y_offset":-1254.1}
(0, 2, 0, 0)


In [96]:
walls = {
    0: (0, 0, 0, 0),
    1: (),
}

In [ ]:
def dfs(maze, v):
    data = sensor_data()
    f, r, l, b = get_walls(data)
    w = l + 
    if f